In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../frameaxis')
from semaxis import CoreUtil
from semaxis import SemAxis
import pandas as pd
import logging
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [4]:
df = pd.read_csv("restaurant.tsv", sep="\t")

In [5]:
# %%time
sa = SemAxis(CoreUtil.load_embedding("../data/embeddings/glove.840B.300d.txt", is_binary=False), # ../data/embeddings/glove.840B.300d.w2vformat.txt
               axes_str=CoreUtil.load_wordnet_antonyms_axes())

2025-04-28 22:37:59 gensim.models.keyedvectors INFO     loading projection weights from ../data/embeddings/glove.840B.300d.txt
2025-04-28 22:39:48 gensim.models.keyedvectors WARNING  duplicate word '����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������' in word2vec file, ignoring all but first
2025-04-28 22:59:31 gensim.utils INFO     KeyedVectors lifecycle event {'msg': 'loaded (2196017, 300) matrix of type float32 from ../data/embeddings/glove.840B.300d.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2025-04-28T22:59:31.910260', 'gensim': '4.1.2', 'python': '3.8.0 (tags/v3.8.0:fa919fd, Oct 14 2019, 19:37:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'load_word2ve

In [6]:
# pickle.dump(sa, open('sa_object.p', 'wb'))

In [7]:
# import pickle
# sa = pickle.load(open('sa_object.p', 'rb'))

In [15]:
df.head()

,sid,text,aspect,sentiment
0,3121,But the staff was so horrible to us.,service,negative
1,2777,"To be completely fair, the only redeeming fact...",food,positive
2,2777,"To be completely fair, the only redeeming fact...",anecdotes/miscellaneous,negative
3,1634,"The food is uniformly exceptional, with a very...",food,positive
4,2534,Where Gabriela personaly greets you and recomm...,service,positive


In [9]:
df.shape

(3713, 4)

In [16]:
df = df.dropna()

In [17]:
df.shape

(3713, 4)

In [21]:
COLUMNS = [str(c) for c in sorted(sa.axes.keys()) if len(c) == 2]

with open("big_table_by_average.tsv", "w") as fo_a:
    fo_a.write("{}\t{}\t{}\t{}\t{}\n".format("sid", "text", "aspect", "sentiment", "\t".join(COLUMNS)))
    for loop_index, (row_index, row) in enumerate(df.iterrows()):
        to_filter = set([row['aspect'], row['aspect']+'s'])

        if loop_index % (1000) == 0:
            logger.info(loop_index)
        try:
            mean = sa.compute_document_mean_with_tf([row['text']], min_freq = 1, to_filter = to_filter)[0]
        except Exception as e:
            print(f"Error en fila {loop_index}: {e}")
            continue

        fo_a.write("{}\t{}\t{}\t{}\t{}\n".format(row['sid'], row['text'], row['aspect'], row['sentiment'], "\t".join([str(v) for v in mean])))
#         break

2025-03-22 20:34:14 __main__     INFO     0


Error en fila 236: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_1/Sum}} = Sum[T=DT_DOUBLE, Tidx=DT_INT32, keep_dims=true](l2_normalize_1/Square, l2_normalize_1/Sum/reduction_indices)' with input shapes: [0], [] and with computed input tensors: input[1] = <1>.
Error en fila 265: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_1/Sum}} = Sum[T=DT_DOUBLE, Tidx=DT_INT32, keep_dims=true](l2_normalize_1/Square, l2_normalize_1/Sum/reduction_indices)' with input shapes: [0], [] and with computed input tensors: input[1] = <1>.
Error en fila 324: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_1/Sum}} = Sum[T=DT_DOUBLE, Tidx=DT_INT32, keep_dims=true](l2_normalize_1/Square, l2_normalize_1/Sum/reduction_indices)' with input shapes: [0], [] and with computed input tensors: input[1] = <1>.
Error en fila 550: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_

2025-03-22 20:34:56 __main__     INFO     1000


Error en fila 1216: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_1/Sum}} = Sum[T=DT_DOUBLE, Tidx=DT_INT32, keep_dims=true](l2_normalize_1/Square, l2_normalize_1/Sum/reduction_indices)' with input shapes: [0], [] and with computed input tensors: input[1] = <1>.


2025-03-22 20:35:36 __main__     INFO     2000


Error en fila 2041: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_1/Sum}} = Sum[T=DT_DOUBLE, Tidx=DT_INT32, keep_dims=true](l2_normalize_1/Square, l2_normalize_1/Sum/reduction_indices)' with input shapes: [0], [] and with computed input tensors: input[1] = <1>.
Error en fila 2886: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_1/Sum}} = Sum[T=DT_DOUBLE, Tidx=DT_INT32, keep_dims=true](l2_normalize_1/Square, l2_normalize_1/Sum/reduction_indices)' with input shapes: [0], [] and with computed input tensors: input[1] = <1>.


2025-03-22 20:36:22 __main__     INFO     3000


# second moment with corpus mean

In [ ]:
# test
import numpy as np
df_corpus = pd.read_csv("big_table_by_average.tsv", sep="\t")

df_corpus.head()  # Para inspeccionar los primeros datos


(3705, 1625)
    sid                                               text  \
0  3121               But the staff was so horrible to us.   
1  2777  To be completely fair, the only redeeming fact...   
2  2777  To be completely fair, the only redeeming fact...   
3  1634  The food is uniformly exceptional, with a very...   
4  2534  Where Gabriela personaly greets you and recomm...   

                    aspect sentiment  ('abaxial', 'adaxial')  \
0                  service  negative                0.011752   
1                     food  positive               -0.006136   
2  anecdotes/miscellaneous  negative               -0.007172   
3                     food  positive                0.007961   
4                  service  positive               -0.038334   

   ('able', 'unable')  ('abnormal', 'normal')  ('aboral', 'oral')  \
0           -0.070756                0.114546            0.211094   
1           -0.075580                0.068166            0.209018   
2           -0.077399 

In [ ]:
# para cada microframe promedia el bias a lo largo de los textos
corpus_mean = np.mean(df_corpus.drop(
    columns=[c for c in df_corpus.columns if '(' not in c]).values, axis=0)
corpus_mean

array([ 0.01595724, -0.12854151,  0.14096514, ...,  0.0613498 ,
       -0.12539983,  0.04118197])

In [25]:
COLUMNS = [str(c) for c in sorted(sa.axes.keys()) if len(c) == 2]

with open("big_table_by_second_moment_with_corpus_mean.tsv", "w") as fo_k:
    fo_k.write("{}\t{}\t{}\t{}\t{}\n".format("sid", "text", "aspect", "sentiment", "\t".join(COLUMNS)))
#     df_corpus = pd.read_csv("big_table_200k_by_average.tsv", sep="\t")
#     corpus_mean = np.mean(df_corpus.drop(
#         columns=[c for c in df_corpus.columns if '(' not in c]).as_matrix(), axis=0)
    for loop_index, (row_index, row) in enumerate(df.iterrows()):
        to_filter = set([row['aspect'], row['aspect']+'s'])
        
        if loop_index % (1000) == 0:
            logger.info(loop_index)
        try:
            sm = sa.compute_document_second_moment_with_tf([row['text']], corpus_mean, min_freq = 1, to_filter = to_filter)  
        except Exception as e:
            print(f"Error en fila {loop_index}: {e}")
            continue

        fo_k.write("{}\t{}\t{}\t{}\t{}\n".format(row['sid'], row['text'], row['aspect'], row['sentiment'], "\t".join([str(v) for v in sm[0]])))
#         break

2025-03-22 20:41:21 __main__     INFO     0


Error en fila 236: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_1/Sum}} = Sum[T=DT_DOUBLE, Tidx=DT_INT32, keep_dims=true](l2_normalize_1/Square, l2_normalize_1/Sum/reduction_indices)' with input shapes: [0], [] and with computed input tensors: input[1] = <1>.
Error en fila 265: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_1/Sum}} = Sum[T=DT_DOUBLE, Tidx=DT_INT32, keep_dims=true](l2_normalize_1/Square, l2_normalize_1/Sum/reduction_indices)' with input shapes: [0], [] and with computed input tensors: input[1] = <1>.
Error en fila 324: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_1/Sum}} = Sum[T=DT_DOUBLE, Tidx=DT_INT32, keep_dims=true](l2_normalize_1/Square, l2_normalize_1/Sum/reduction_indices)' with input shapes: [0], [] and with computed input tensors: input[1] = <1>.
Error en fila 550: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_

2025-03-22 20:42:05 __main__     INFO     1000


Error en fila 1216: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_1/Sum}} = Sum[T=DT_DOUBLE, Tidx=DT_INT32, keep_dims=true](l2_normalize_1/Square, l2_normalize_1/Sum/reduction_indices)' with input shapes: [0], [] and with computed input tensors: input[1] = <1>.


2025-03-22 20:42:56 __main__     INFO     2000


Error en fila 2041: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_1/Sum}} = Sum[T=DT_DOUBLE, Tidx=DT_INT32, keep_dims=true](l2_normalize_1/Square, l2_normalize_1/Sum/reduction_indices)' with input shapes: [0], [] and with computed input tensors: input[1] = <1>.
Error en fila 2886: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node l2_normalize_1/Sum}} = Sum[T=DT_DOUBLE, Tidx=DT_INT32, keep_dims=true](l2_normalize_1/Square, l2_normalize_1/Sum/reduction_indices)' with input shapes: [0], [] and with computed input tensors: input[1] = <1>.


2025-03-22 20:43:45 __main__     INFO     3000
